## Model metrics

In [6]:
import tensorboard_reducer as tbr
import matplotlib.pyplot as plt

In [7]:
LOG_PATH = "lightning_logs/"
def get_metrics(model_folder):
    log_data = tbr.load_tb_events([LOG_PATH+model_folder], verbose=True) #,strict_tags=False,strict_steps=False,handle_dup_steps='keep-first')
    epochs = log_data['epoch'].values
    train_loss_epoch = log_data['train_loss_epoch'].values
    train_acc_epoch = log_data['train_acc_epoch'].values
    val_loss = log_data['val_loss'].values
    val_acc = log_data['val_acc'].values
    steps = log_data.keys()
    print(len(steps))
    for key in log_data:
        print(key)
        print(len(log_data[key]))
    return steps,epochs, train_loss_epoch,train_acc_epoch,val_loss,val_acc


In [20]:
from collections import defaultdict
from typing import Literal, get_args

import pandas as pd
from tqdm import tqdm

from tensorboard_reducer.event_loader import EventAccumulator

def get_metrics2(model_folder):
    dirname = LOG_PATH+model_folder
    accumulator =     EventAccumulator(dirname).Reload()
    load_dict = defaultdict(list)
    for tag in accumulator.scalar_tags:
        # accumulator.Scalars() returns columns 'step', 'wall_time', 'value'
        df_scalar = pd.DataFrame(accumulator.Scalars(tag)).set_index("step")
        df_scalar = df_scalar.drop(columns="wall_time")
        load_dict[tag].append(df_scalar)
    out_dict: dict[str, pd.DataFrame] = {}

    out_dict = {
        key: pd.concat(lst, join="inner", axis=1) for key, lst in load_dict.items()
    }
    for key in out_dict:
        print(key)
        print(len(out_dict[key]))
    return out_dict


In [21]:
t = get_metrics2('resnet_18_best_subset')
print(t)

train_loss_step
63
train_acc_step
63
epoch
183
val_loss
60
val_acc
60
train_loss_epoch
60
train_acc_epoch
60
{'train_loss_step':          value
step          
49    0.650718
99    0.383621
149   0.318134
199   0.251679
249   0.253420
...        ...
2949  0.222659
2999  0.218136
3049  0.212591
3099  0.220684
3149  0.213404

[63 rows x 1 columns], 'train_acc_step':       value
step       
49     0.74
99     0.93
149    0.96
199    1.00
249    1.00
...     ...
2949   1.00
2999   1.00
3049   1.00
3099   1.00
3149   1.00

[63 rows x 1 columns], 'epoch':       value
step       
49      0.0
52      0.0
52      0.0
99      1.0
105     1.0
...     ...
3126   58.0
3126   58.0
3149   59.0
3179   59.0
3179   59.0

[183 rows x 1 columns], 'val_loss':          value
step          
52    1.235157
105   0.672453
158   0.397136
211   0.398125
264   0.397894
317   0.406826
370   0.396089
423   0.415718
476   0.403996
529   0.419049
582   0.413731
635   0.442042
688   0.414572
741   0.411839
794   0.4143

In [10]:
steps, epochs, train_loss,train_acc,val_loss,val_acc = get_metrics('resnet_18_best_subset')
fig = plt.figure(figsize=(8,4))
ax1 = fig.add_subplot(221) 
ax1.plot(steps,train_loss,linestyle='-', c='orange')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Train Loss')

ax2 = fig.add_subplot(222) 
ax2.plot(steps,val_loss,linestyle='-', c='violet')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Validation Loss')

ax3 = fig.add_subplot(223) 
ax3.plot(steps,train_acc,linestyle='-', c='red')
ax3.set_xlabel('Epochs')
ax3.set_ylabel('Train Accuracy')

ax4 = fig.add_subplot(224) 
ax4.plot(steps,val_acc,linestyle='-', c='green')
ax4.set_xlabel('Epochs')
ax4.set_ylabel('Validation Accuracy')
plt.tight_layout()
plt.show()

Reading tags:   0%|          | 0/1 [00:00<?, ?it/s]

Reading tags:   0%|          | 0/1 [00:00<?, ?it/s]


ValueError: Tag 'epoch' from run directory 'lightning_logs/resnet_18_best_subset' contains duplicate steps. Please make sure your data wasn't corrupted. If this is expected/you want to proceed anyway, specify how to handle duplicate values recorded for the same tag and step in a single run by passing --handle-dup-steps to the CLI or handle_dup_steps='keep-first'|'keep-last'|'mean' to the Python API. This will keep the first/last occurrence of duplicate steps or take their mean.